In [3]:
from newspaper import Article

import requests
import feedparser
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime, timezone


In [20]:
def extract_article(url):
    try:
        article = Article(url, language="fr")
        article.download()
        article.parse()
        return {
            "url": url,
            "title": article.title,
            "text": article.text,
            "publish_date": article.publish_date if article.publish_date else ""
        }
    except Exception as e:
        print(f"❌ Erreur avec {url} : {e}")
        return None

# ▶️ Lire les URLs depuis le fichier
with open("urls_franceinfo.txt", "r", encoding="utf-8") as f:
    urls = [line.strip() for line in f if line.strip()]

# ▶️ Extraction en boucle
articles_data = []
for i, url in enumerate(urls):
    print(f"[{i+1}/{len(urls)}] Extraction de : {url}")
    data = extract_article(url)
    if data and len(data["text"]) > 300:  # filtrer les articles trop courts
        articles_data.append(data)
    time.sleep(1)  # pause douce pour éviter les blocages

# ▶️ Enregistrement dans un CSV
df = pd.DataFrame(articles_data)
df.to_csv("articles_franceinfo_extraits.csv", index=False, encoding="utf-8")
print(f"✅ {len(df)} articles sauvegardés dans 'articles_franceinfo_extraits.csv'")

[1/145] Extraction de : https://www.francetvinfo.fr/economie/automobile/maires-en-colere-des-methodes-radicales-pour-faire-baisser-la-vitesse_7282128.html
[2/145] Extraction de : https://www.francetvinfo.fr/economie/budget/budget-f-bayrou-demande-un-effort-aux-francais_7277373.html
[3/145] Extraction de : https://www.francetvinfo.fr/economie/budget/budget-francois-bayrou-demande-aux-francais-de-faire-des-efforts_7276326.html
[4/145] Extraction de : https://www.francetvinfo.fr/economie/emploi/metiers/agriculture/pont-de-l-ascension-les-deputes-ne-siegeront-pas-les-30-et-31-mai-pour-examiner-le-texte-visant-a-lever-les-contraintes-dans-l-agriculture_7265805.html
[5/145] Extraction de : https://www.francetvinfo.fr/economie/emploi/metiers/agriculture/temoignages-c-est-une-vraie-coupe-budgetaire-les-professeurs-de-lycees-agricoles-protestent-contre-les-suppressions-de-postes-prevues_7263309.html
[6/145] Extraction de : https://www.francetvinfo.fr/economie/entreprises/qu-est-ce-que-le-devoir

In [ ]:

def fetch_articles_from_franceinfo(rss_url: str) -> list[dict]:
    feed = feedparser.parse(rss_url)
    print(feed)
    articles = []

    for entry in feed.entries:
        print(entry)
        article = {
            "title": entry.title,
            "summary": entry.get("summary", ""),
            "url": entry.link,
            "published_at": entry.get("published", datetime.now(timezone.utc).isoformat()),
            "fetched_at": datetime.now(timezone.utc).isoformat(),
            "source": "franceinfo",
            "language": "fr"
        }
        articles.append(article)

    return articles

url = 'https://www.franceinfo.fr/titres.rss'

print(fetch_articles_from_franceinfo(url))

In [27]:
import json
import logging
from google.cloud import storage

# Configure le logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

def archive_blob(blob, folder_prefix):
    client = storage.Client()
    bucket = blob.bucket

    blob_name = blob.name
    if blob_name.startswith(folder_prefix + "archived_"):
        return

    filename = blob_name[len(folder_prefix):]
    new_name = folder_prefix + "archived_" + filename

    bucket.copy_blob(blob, bucket, new_name)
    blob.delete()

    logger.info(f"Archivé : {blob_name} → {new_name}")

def collect_articles_from_gcs(bucket_name, folder_prefix):
    """
    Récupère tous les articles JSON depuis un dossier GCS, 
    en éliminant les doublons basés sur l'ID.
    
    Retourne :
        List[dict] : articles uniques
    """
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=folder_prefix)

    articles_by_id = {}
    fichiers_traites = 0

    for blob in blobs:
        if blob.name.endswith('/'):
            continue

        relative_name = blob.name[len(folder_prefix):]
        if relative_name.startswith("archived_"):
            continue

        try:
            content = blob.download_as_text()
            data = json.loads(content)

            successfully_parsed = False

            if isinstance(data, list):
                for article in data:
                    if isinstance(article, dict):
                        article_id = article.get("id")
                        if article_id and article_id not in articles_by_id:
                            articles_by_id[article_id] = article
                            successfully_parsed = True
            elif isinstance(data, dict):
                article_id = data.get("id")
                if article_id and article_id not in articles_by_id:
                    articles_by_id[article_id] = data
                    successfully_parsed = True

            if successfully_parsed:
                # archive_blob(blob, folder_prefix)
                fichiers_traites += 1

        except Exception as e:
            logger.warning(f"Erreur de parsing JSON dans {blob.name} : {e}")

    logger.info(f"{fichiers_traites} fichiers traités et archivés.")
    logger.info(f"{len(articles_by_id)} articles uniques collectés.")

    return list(articles_by_id.values())


In [20]:
(collect_articles_from_gcs('clarifai-news-bucket', 'rss_articles'))

[{'id': '76814b5ccc2238dc3cd000e204c362a2',
  'title': 'MaPrimRénov\', zones à faibles émissions... Emmanuel Macron déplore "une erreur historique" après la suppression de plusieurs mesures écologiques',
  'summary': '"Je ne suis pas content de ce que j\'ai pu voir", déclare le président de la République, dans un entretien à la presse régionale.',
  'url': 'https://www.franceinfo.fr/environnement/maprimrenov-suppression-des-zones-a-faibles-emissions-emmanuel-macron-denonce-le-detricotage-de-plusieurs-mesures-ecologiques_7298379.html#xtor=RSS-3-[lestitres]',
  'published_at': '2025-06-07T18:23:19+00:00',
  'fetched_at': '2025-06-07T20:41:20+00:00',
  'source': 'franceinfo',
  'language': 'fr'},
 {'id': 'd959b1d47d3bb08c58f23d07fb154f27',
  'title': 'Roland-Garros 2025 : revivez la finale spectaculaire remportée par Coco Gauff face à Aryna Sabalenka',
  'summary': "L'Américaine de 21 ans a remporté son deuxième titre du Grand Chelem, le premier à Roland-Garros, samedi.",
  'url': 'https:

In [28]:
from newspaper import Article
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

def extract_article_content(url):
    """Extrait le texte principal d'un article à partir de son URL."""
    try:
        article = Article(url, language="fr")
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        logger.warning(f"❌ Erreur avec {url} : {e}")
        return None

def enrich_articles_with_content(articles, limit=None):
    """
    Enrichit une liste d'articles en ajoutant le champ 'content' via scraping.

    Args:
        articles (List[dict]): Liste d'articles avec au moins le champ 'url'.
        limit (int, optional): Pour restreindre le nombre traité.

    Returns:
        List[dict]: Liste enrichie avec le champ 'content'.
    """
    enriched = []
    to_process = articles if limit is None else articles[:limit]

    for article in to_process:
        url = article.get("url")
        if not url:
            continue

        logger.info(f"Scraping : {url}")
        content = extract_article_content(url)

        if content:
            enriched_article = article.copy()
            enriched_article["content"] = content
            enriched.append(enriched_article)

    logger.info(f"{len(enriched)} articles enrichis avec le champ 'content'.")
    return enriched

In [31]:
if __name__ == "__main__":
    articles = [
        {"id": "abc1", "url": "https://www.franceinfo.fr/roland-garros/videos/video-roland-garros-2025-revivez-la-finale-spectaculaire-remportee-par-coco-gauff-face-a-aryna-sabalenka_7298340.html#xtor=RSS-3-[lestitres]"},
        {"id": "abc2", "url": "https://www.franceinfo.fr/environnement/maprimrenov-suppression-des-zones-a-faibles-emissions-emmanuel-macron-denonce-le-detricotage-de-plusieurs-mesures-ecologiques_7298379.html#xtor=RSS-3-[lestitres]"}
    ]

    result = enrich_articles_with_newspaper(articles)
    for a in result:
        print(a)
        # print(a["content"][:300])

{'id': 'abc1', 'url': 'https://www.franceinfo.fr/roland-garros/videos/video-roland-garros-2025-revivez-la-finale-spectaculaire-remportee-par-coco-gauff-face-a-aryna-sabalenka_7298340.html#xtor=RSS-3-[lestitres]', 'content': "L'Américaine de 21 ans a remporté son deuxième titre du Grand Chelem, le premier à Roland-Garros, samedi.\n\nDes larmes de joie pour Coco Gauff, de détresse pour Aryna Sabalenka. Les deux joueuses visaient un premier titre à Roland-Garros, et c'est finalement la jeune Américaine de 21 ans qui l'a emporté (6-7 [5-7], 6-2, 6-4), samedi 7 juin. Elle a notamment profité des trop nombreuses fautes directes (70) de la Biélorusse pour renverser un match qu'elle avait mal débuté.\n\nFrustrée de son niveau, et du vent qui soufflait sur le court, Aryna Sabalenka s'est agacée, ce qui n'a pas arrangé son niveau de jeu, face à une Coco Gauff très concentrée et impassible, trois ans après sa finale perdue sur la même terre battue contre la Polonaise et quadruple lauréate de la C